<a href="https://colab.research.google.com/github/PegasusArc/stock_predictions_using_lstm/blob/main/stock_market_analysis_prediction_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas_datareader.data import DataReader
import warnings
from datetime import datetime
import yfinance as yf

warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('./AAPL.csv')
print(data.shape)
print(data.sample(7))

In [ ]:
data.info()

In [ ]:
# date vs open
# date vs close
plt.figure(figsize=(60, 30))
plt.subplot(2, 2, 1)
plt.plot(data['Date'], data['Close'], c="r", label="Close", marker="+")
plt.plot(data['Date'], data['Open'], c="g", label="Open", marker="^")
plt.title("AAPL")
plt.legend()
plt.tight_layout()

In [ ]:
plt.figure(figsize=(60, 30))
plt.subplot(2, 2, 1)
plt.plot(data['Date'], data['Volume'], c='purple', marker='*')
plt.title(f"AAPL Volume")
plt.tight_layout()

In [ ]:
prediction_range = data.loc[(data['Date'] > datetime(2013,1,1)) & (data['Date']<datetime(2020,1,1))]
plt.figure(figsize=(60, 30))
plt.subplot(2, 2, 1)

plt.plot(data['Date'], data['Close'])
plt.xlabel("Date")
plt.ylabel("Close")
plt.title("AAPL Stock Prices")
plt.show()

In [ ]:
close_data = data.filter(['Close'])
dataset = close_data.values
training = int(np.ceil(len(dataset) * .95))
print(training)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

train_data = scaled_data[0:int(training), :]
# prepare feature and labels
x_train = []
y_train = []

for i in range(60, len(train_data)):
	x_train.append(train_data[i-60:i, 0])
	y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(units=64,
					return_sequences=True,
					input_shape=(x_train.shape[1], 1)))
model.add(keras.layers.LSTM(units=64))
model.add(keras.layers.Dense(32))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',
			loss='mean_squared_error')
history = model.fit(x_train,
					y_train,
					epochs=10)

In [ ]:
test_data = scaled_data[training - 60:, :]
x_test = []
y_test = dataset[training:, :]
for i in range(60, len(test_data)):
	x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# predict the testing data
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# evaluation metrics
mse = np.mean(((predictions - y_test) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

In [ ]:
train = data[:training]
test = data[training:]
test['Predictions'] = predictions

plt.figure(figsize=(60, 30))
plt.subplot(2, 2, 1)
plt.plot(train['Date'], train['Close'])
plt.plot(test['Date'], test[['Close', 'Predictions']])
plt.title('Apple Stock Close Price')
plt.xlabel('Date')
plt.ylabel("Close")
plt.legend(['Train', 'Test', 'Predictions'])